In [1]:
def readin(filename):
    with open(filename) as f:
        lines = f.readlines()

    #something to remove \n 
    str_lines = []
    for l in lines:
        if l[-1:]=="\n":
            str_lines.append(l[0:-1])
        else: 
            str_lines.append(l)

    #check all was fine
    # print(lines[-2:])
    # print(str_lines[-2:])
    return str_lines

elf_data = readin("Day15input.txt")

# --- Day 15: Beacon Exclusion Zone ---

You feel the ground rumble again as the distress signal leads you to a large network of subterranean tunnels. You don't have time to search them all, but you don't need to: your pack contains a set of deployable sensors that you imagine were originally built to locate lost Elves.

The sensors aren't very powerful, but that's okay; your handheld device indicates that you're close enough to the source of the distress signal to use them. You pull the emergency sensor system out of your pack, hit the big button on top, and the sensors zoom off down the tunnels.

Once a sensor finds a spot it thinks will give it a good reading, it attaches itself to a hard surface and begins monitoring for the nearest signal source beacon. Sensors and beacons always exist at integer coordinates. Each sensor knows its own position and can determine the position of a beacon precisely; however, sensors can only lock on to the one beacon closest to the sensor as measured by the Manhattan distance. (There is never a tie where two beacons are the same distance to a sensor.)

It doesn't take long for the sensors to report back their positions and closest beacons (your puzzle input). For example:


In [2]:
test_data = ["Sensor at x=2, y=18: closest beacon is at x=-2, y=15",
"Sensor at x=9, y=16: closest beacon is at x=10, y=16",
"Sensor at x=13, y=2: closest beacon is at x=15, y=3",
"Sensor at x=12, y=14: closest beacon is at x=10, y=16",
"Sensor at x=10, y=20: closest beacon is at x=10, y=16",
"Sensor at x=14, y=17: closest beacon is at x=10, y=16",
"Sensor at x=8, y=7: closest beacon is at x=2, y=10",
"Sensor at x=2, y=0: closest beacon is at x=2, y=10",
"Sensor at x=0, y=11: closest beacon is at x=2, y=10",
"Sensor at x=20, y=14: closest beacon is at x=25, y=17",
"Sensor at x=17, y=20: closest beacon is at x=21, y=22",
"Sensor at x=16, y=7: closest beacon is at x=15, y=3",
"Sensor at x=14, y=3: closest beacon is at x=15, y=3",
"Sensor at x=20, y=1: closest beacon is at x=15, y=3"]

In [3]:
def parse_data(data):
    returnData = []
    for d in data:
        pair = d.split(":")
        sensor = pair[0].split(",")
        d1 = eval(sensor[0].removeprefix("Sensor at x="))
        d2 = eval(sensor[1].removeprefix(" y="))
        beacon = pair[1].split(",")
        d3 = eval(beacon[0].removeprefix(" closest beacon is at x="))
        d4 = eval(beacon[1].removeprefix(" y="))
        manhattan = abs(d3-d1) + abs(d4-d2)
        upper = d2+manhattan
        lower = d2-manhattan
        returnData.append([d1, d2, d3, d4, manhattan, upper, lower])
        #print(d1, d2, d3, d4)
    return returnData

parsed_test_data = parse_data(test_data)
parsed_elf_data = parse_data(elf_data)


So, consider the sensor at 2,18; the closest beacon to it is at -2,15. For the sensor at 9,16, the closest beacon to it is at 10,16.

Drawing sensors as S and beacons as B, the above arrangement of sensors and beacons looks like this:

                1    1    2    2
        0    5    0    5    0    5
    0 ....S.......................
    1 ......................S.....
    2 ...............S............
    3 ................SB..........
    4 ............................
    5 ............................
    6 ............................
    7 ..........S.......S.........
    8 ............................
    9 ............................
    10 ....B.......................
    11 ..S.........................
    12 ............................
    13 ............................
    14 ..............S.......S.....
    15 B...........................
    16 ...........SB...............
    17 ................S..........B
    18 ....S.......................
    19 ............................
    20 ............S......S........
    21 ............................
    22 .......................B....


In [4]:
def get_grid_dim(pd):
    min_x = pd[0][0]
    max_x = pd[0][0]
    # min_y = pd[0][1]
    # max_y = pd[0][1]
    for j in pd:
        # print(i)
        # print(j)
        # print(j[0])
        # print(j[1])
        # min_y = min(j[1], j[3], min_y)
        # max_y = max(j[1], j[3], max_y)
        min_x = min(j[0], j[2], min_x)
        max_x = max(j[0], j[2], max_x)

    return (min_x, max_x)


This isn't necessarily a comprehensive map of all beacons in the area, though. Because each sensor only identifies its closest beacon, if a sensor detects a beacon, you know there are no other beacons that close or closer to that sensor. There could still be beacons that just happen to not be the closest beacon to any sensor. Consider the sensor at 8,7:

                1    1    2    2
        0    5    0    5    0    5
    -2 ..........#.................
    -1 .........###................
    0 ....S...#####...............
    1 .......#######........S.....
    2 ......#########S............
    3 .....###########SB..........
    4 ....#############...........
    5 ...###############..........
    6 ..#################.........
    7 .#########S#######S#........
    8 ..#################.........
    9 ...###############..........
    10 ....B############...........
    11 ..S..###########............
    12 ......#########.............
    13 .......#######..............
    14 ........#####.S.......S.....
    15 B........###................
    16 ..........#SB...............
    17 ................S..........B
    18 ....S.......................
    19 ............................
    20 ............S......S........
    21 ............................
    22 .......................B....

This sensor's closest beacon is at 2,10, and so you know there are no beacons that close or closer (in any positions marked #).


In [5]:
def get_sensors_or_beacons(parsed_data, min_x, s_or_b):
    coords = {}
    if s_or_b == "S":
        key = 1
    else:
        key = 3
    for p in parsed_data:
    # sensor_x = sensor[0]-min_x
    # sensor_y = sensor[1]  # no need to normalise to 0 index
    # beacon_x = sensor[2]-min_x
    # beacon_y = sensor[3]
    # s0 = s0+1
    # print("Checking sensor ", s0, " of ", s)
        x = p[key-1] - min_x
        if p[key] in coords.keys():
            if key == 1 or (key == 3 and x not in coords[p[key]]):
                coords[p[key]].append(x)
        else:
            coords[p[key]] = [x]
    return coords
        


In [6]:
def setup_row_from_scratch(new_line, sensors_in_row, beacons_in_row, min_x):
    #print(len(new_line))
    # s = len(parsed_data)
    # s0 = 0
    # # print("Empty row set up, checking ", s, " sensors")
    if sensors_in_row is not None:
        for sensor in sensors_in_row:
        # sensor_x = sensor[0]-min_x
        # sensor_y = sensor[1]  # no need to normalise to 0 index
        # beacon_x = sensor[2]-min_x
        # beacon_y = sensor[3]
        # s0 = s0+1
        # print("Checking sensor ", s0, " of ", s)
            new_line[sensor-min_x] = "S"
            # print("sensor added at ", sensor_x, row_num)
    if beacons_in_row is not None:
        for beacon in beacons_in_row:
            new_line[beacon-min_x] =  "B"
            # print("beacon added at ", beacon_x, row_num)
    # print("Finished, returning row")
    return new_line


In [7]:
def check_row_from_scratch(row, row_no, min_x, max_x, parsed_data):
    coords = []
    for sensor in parsed_data:
        if row_no in range(sensor[6], sensor[5]+1): #shoudl this be sensor[5]+1?
            x_range = sensor[4] - abs(sensor[1] - row_no) 
            x_left = sensor[0]-min_x - x_range
            x_left = max(x_left, 0)
            x_right = sensor[0]-min_x + x_range
            x_right = min(x_right, (len(row)-1))
            if (x_left, x_right) not in coords:
                coords.append((x_left, x_right))
            #print("Checking sensor at ", sensor_x, sensor_y, " over range ", x_left, x_right)
    coords.sort()
    current_left = coords[0][0]
    current_right = coords[0][1]
    na = 0#row.count("S") #+ row.count("B")
    
    if current_left>min_x:
        (poss, poss_found) = check_for_availables(row, min_x, current_left, min_x, -1, False)
    else:
        poss = -1
        poss_found = False
        
    end_reached = False

    i = -1
    while i<(len(coords)-1) and end_reached == False:
        i = i+1
        #print(coords[i], i, len(coords))
        if coords[i][0] in range(current_left, current_right+1):  #if x in current coords
            if coords[i][1] not in range(current_left, current_right+1):    
                current_right = coords[i][1]  # extend current coords
        else:  #assume x greater than current, start a new coord, process the last one
            if not poss_found:
                (poss, poss_found) = check_for_availables(row, current_right+1, coords[i][0], min_x, poss, poss_found)
            na=na+current_right-current_left
            current_left = coords[i][0]
            current_right = coords[i][1]
        if current_right == max_x:
            end_reached = True
            na=na+current_right-current_left
    if not poss_found and (current_right-current_left)<(max_x-min_x):
        (poss, poss_found) = check_for_availables(row, current_right+1, len(row)+min_x, min_x, poss, poss_found)
    if not end_reached:
        na=na+current_right-current_left
    return (na, poss)



    

def check_for_availables(row, start, end, min_x, poss, poss_found):
    temp_slice = row[start-min_x:end-min_x]
    if temp_slice.__contains__("."): #check for available positions 
        poss = temp_slice.index(".")+(start-min_x)
        poss_found = True
    return poss, poss_found


None of the detected beacons seem to be producing the distress signal, so you'll need to work out where the distress beacon is by working out where it isn't. For now, keep things simple by counting the positions where a beacon cannot possibly be along just a single row.

So, suppose you have an arrangement of beacons and sensors like in the example above and, just in the row where y=10, you'd like to count the number of positions a beacon cannot possibly exist. The coverage from all sensors near that row looks like this:

                    1    1    2    2
        0    5    0    5    0    5
    9 ...#########################...
    10 ..####B######################..
    11 .###S#############.###########.


In [8]:
def explore_row(row_num, parsed_data, min_x, max_x, part, sensors, beacons, new_line):
    # print("get grid_dim ", min_x, max_x, " now starting setup row ", row_num)
    nl = new_line.copy()   # having issues here
    if row_num not in sensors.keys():
        sensors_in_row = None
    else: 
        sensors_in_row = sensors[row_num]
    if row_num not in beacons.keys():
        beacons_in_row = None
    else: 
        beacons_in_row = beacons[row_num]
    row = setup_row_from_scratch(nl, sensors_in_row, beacons_in_row, min_x)
    # print("Row set up length ", len(row), " starting to check row ", row_num)
    return check_row_from_scratch(row, row_num, min_x, max_x, parsed_data)

In [9]:
def test(parsed_data, part, is_test, row_num):
    if part == 1:
        (min_x, max_x) = get_grid_dim(parsed_data)
        min_x = min(min_x, -3000000)
        max_x = max(max_x, 8000000)
    elif is_test: 
        min_x = 0
        max_x = 20
    else: 
        min_x = 0
        max_x = 4000000
    sensors = get_sensors_or_beacons(parsed_data, min_x, "S")
    beacons = get_sensors_or_beacons(parsed_data, min_x, "B")
    new_line = []
    for x in range(min_x, max_x+1):    
        new_line.append(".")
    if part == 1:
        return explore_row(row_num, parsed_data, min_x, max_x, part, sensors, beacons, new_line)
    else:
        return look_for_tuning_freq(parsed_data, is_test, min_x, max_x, part, sensors, beacons, new_line)

def look_for_tuning_freq(parsed_data, is_test, min_x, max_x, part, sensors, beacons, new_line):
    y = 0
    if part==2 and is_test == False:
        y = 309000
    found = False
    limit = 20
    if not is_test:
        limit = 4000000
    notavailable3 = max_x
    possible3 = -1
    while y<limit and found is False:
        (notavailable3, possible3) = explore_row(y, parsed_data, min_x, max_x, part, sensors, beacons, new_line)
        if y%1000 == 0:
            print(y, notavailable3, possible3, min_x, max_x)
        if possible3 > -1 and possible3-min_x <= max_x:
            found = True
        else: 
            y = y+1
    print(notavailable3, possible3, y, min_x, max_x)
    return calc_tuning_frequency(possible3,y)

def calc_tuning_frequency(x, y):
    return ((x*4000000)+y)

In [10]:
print(test(parsed_test_data, 1, True, 10))

(26, 0)



In this example, in the row where y=10, there are 26 positions where a beacon cannot be present.

Consult the report from the sensors you just deployed. In the row where y=2000000, how many positions cannot contain a beacon?

In [11]:
print(test(parsed_elf_data, 1, False, 2000000))

(5525990, 0)


# --- Part Two ---

Your handheld device indicates that the distress signal is coming from a beacon nearby. The distress beacon is not detected by any sensor, but the distress beacon must have x and y coordinates each no lower than 0 and no larger than 4000000.

To isolate the distress beacon's signal, you need to determine its tuning frequency, which can be found by multiplying its x coordinate by 4000000 and then adding its y coordinate.

In the example above, the search space is smaller: instead, the x and y coordinates can each be at most 20. With this reduced search area, there is only a single position that could have a beacon: x=14, y=11. The tuning frequency for this distress beacon is 56000011.

Find the only possible position for the distress beacon. What is its tuning frequency?



In [12]:
print(test(parsed_test_data, 2, True, -1))
#beacon: x=14, y=11. The tuning frequency for this distress beacon is 56000011.

0 20 -1 0 20
18 14 11 0 20
56000011


In [13]:
def calc(num_sec_for_100, mult):
    secs = num_sec_for_100*(mult/100)
    print(secs, " secs")
    mins = secs/60
    print(mins, " mins")
    hours = mins/60
    print(hours, " hours")
    days = hours/24
    print(days, " days")
    years = days/365
    print(years, " years")

calc(2, 4000000)

80000.0  secs
1333.3333333333333  mins
22.22222222222222  hours
0.9259259259259259  days
0.0025367833587011668  years


In [14]:
print(test(parsed_elf_data, 2, False, -1))
#            print(y, notavailable3, possible3, min_x, max_x)


309000 4000000 -1 0 4000000
310000 4000000 -1 0 4000000
311000 4000000 -1 0 4000000
312000 4000000 -1 0 4000000
313000 4000000 -1 0 4000000
314000 4000000 -1 0 4000000
315000 4000000 -1 0 4000000
316000 4000000 -1 0 4000000
317000 4000000 -1 0 4000000
318000 4000000 -1 0 4000000
319000 4000000 -1 0 4000000
320000 4000000 -1 0 4000000
321000 4000000 -1 0 4000000
322000 4000000 -1 0 4000000
323000 4000000 -1 0 4000000
324000 4000000 -1 0 4000000
325000 4000000 -1 0 4000000
326000 4000000 -1 0 4000000
327000 4000000 -1 0 4000000
328000 4000000 -1 0 4000000
329000 4000000 -1 0 4000000
330000 4000000 -1 0 4000000
331000 4000000 -1 0 4000000
332000 4000000 -1 0 4000000
333000 4000000 -1 0 4000000
334000 4000000 -1 0 4000000
335000 4000000 -1 0 4000000
336000 4000000 -1 0 4000000
337000 4000000 -1 0 4000000
338000 4000000 -1 0 4000000
339000 4000000 -1 0 4000000
340000 4000000 -1 0 4000000
341000 4000000 -1 0 4000000
342000 4000000 -1 0 4000000
343000 4000000 -1 0 4000000
344000 4000000 -1 0 

KeyboardInterrupt: 

*108000000 is too low*

*got up to 585000*